In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, Reshape, LSTM
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold

In [2]:
data_directory = './data/Train-Test Set 2'

transform = transforms.Compose([
    transforms.Resize((640, 480)),
    # transforms.ToTensor(),
    # transforms.Resize(256),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

custom_dataset = ImageFolder(root=data_directory, transform=transform)

In [3]:
torch.manual_seed(42)

In [4]:
batch_size = 16
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [5]:
model2 = Sequential()

model2.add(Conv2D(16, (3, 3), activation='relu', input_shape=(640, 480, 3)))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))
model2.add(Dropout(0.2))
model2.add(Conv2D(128, (3, 3), activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))
model2.add(Dropout(0.2))
model2.add(Flatten())
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(.7))
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(.5))
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(.3))
model2.add(Dense(4, activation='softmax'))

In [6]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 638, 478, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 319, 239, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 317, 237, 32)      4640      
                                                                 
 batch_normalization (Batch  (None, 317, 237, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 158, 118, 32)      0         
 g2D)                                                            
                                                        

In [8]:
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    './data/Train-Test Set 2/Training',
    target_size=(640, 480),
    batch_size=batch_size,
    class_mode='sparse'
)

validation_generator = test_datagen.flow_from_directory(
    './data/Train-Test Set 2/Validation',
    target_size=(640, 480),
    batch_size=batch_size,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    './data/Train-Test Set 2/Testing',
    target_size=(640, 480),
    batch_size=batch_size,
    class_mode='sparse'
)

Found 169 images belonging to 4 classes.
Found 114 images belonging to 4 classes.
Found 72 images belonging to 4 classes.


In [9]:
model2.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/20


C:\Users\user\AppData\Local\Temp\ipykernel_1068\143086181.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model2.fit_generator(


11/11 [==============================] - 21s 2s/step - loss: 72.3467 - accuracy: 0.3491 - val_loss: 50.5129 - val_accuracy: 0.3947
Epoch 2/20
11/11 [==============================] - 19s 2s/step - loss: 69.5403 - accuracy: 0.2663 - val_loss: 188.4312 - val_accuracy: 0.1491
Epoch 3/20
11/11 [==============================] - 20s 2s/step - loss: 64.0548 - accuracy: 0.3254 - val_loss: 45.2612 - val_accuracy: 0.3509
Epoch 4/20
11/11 [==============================] - 19s 2s/step - loss: 52.8168 - accuracy: 0.3373 - val_loss: 19.3571 - val_accuracy: 0.2018
Epoch 5/20
11/11 [==============================] - 20s 2s/step - loss: 44.5171 - accuracy: 0.2959 - val_loss: 38.0101 - val_accuracy: 0.3421
Epoch 6/20
11/11 [==============================] - 20s 2s/step - loss: 35.3040 - accuracy: 0.3609 - val_loss: 21.7348 - val_accuracy: 0.3947
Epoch 7/20
11/11 [==============================] - 20s 2s/step - loss: 36.9791 - accuracy: 0.2722 - val_loss: 31.5252 - val_accuracy: 0.2544
Epoch 8/20
11/11

In [10]:
model2.save('cnn_only_267mbtrainableparams_20epochs_4035.keras')

In [18]:
from sklearn.metrics import confusion_matrix
import itertools

predictions = model2.predict_generator(test_generator)
y_pred = np.round(predictions).astype(int).flatten()  # Convert probabilities to class labels
# print(y_true, y_pred)
y_true = test_generator.classes
print(y_true)
# cm = confusion_matrix(y_true, y_pred)

# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, cm[i, j],
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')

# plot_confusion_matrix(cm, classes=validation_generator.class_indices.keys())

# plt.show()

C:\Users\user\AppData\Local\Temp\ipykernel_10156\1749941455.py:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model2.predict_generator(test_generator)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3]
